# Adding a New Explainer to GRETEL

In [1]:
import sys
import os
module_path = os.path.abspath(os.path.join('..'))
sys.path.append(module_path)

In [2]:
from src.evaluation.evaluator_manager import EvaluatorManager

config_file_path = module_path + '/examples/config/config_autism_custom-oracle_dummy_explainer.json'
output_file_path = module_path + '/examples/output/asd_custom_oracle/DummyExplainer/results_run-0.json'
output_folder = module_path + '/examples/output/'
stats_folder = module_path + '/examples/stats/'
ex_store_path = module_path + '/data/explainers/'

# Verifying that the paths are valid
(os.path.isfile(config_file_path), os.path.isfile(output_file_path), os.path.isdir(output_folder), os.path.isdir(stats_folder), 
os.path.isdir(ex_store_path))

2023-02-09 22:00:19.042885: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-09 22:00:19.275466: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


(True, False, True, True, True)

## Creating a new Explainer

 Creating the explainer

In [3]:
from src.evaluation.evaluation_metric_base import EvaluationMetric
from src.explainer.explainer_base import Explainer
from src.dataset.dataset_base import Dataset
from src.oracle.oracle_base import Oracle

class DummyExplainer(Explainer):
    """This Dummy Explainer search for the first counterfactual instance in the dataset and returns it"""

    def __init__(self, id, config_dict=None) -> None:
        super().__init__(id, config_dict)
        self._name = 'DummyExplainer'


    def explain(self, instance, oracle: Oracle, dataset: Dataset):
        l_input_inst = oracle.predict(instance)

        # if the method does not find a counterfactual example returns the original graph
        min_counterfactual = instance

        for d_inst in dataset.instances:
            
            l_data_inst = oracle.predict(d_inst)

            if (l_input_inst != l_data_inst):
                min_counterfactual = d_inst

                return min_counterfactual
        
        return min_counterfactual


Creating a custom ExplainerFactory that extends the base class with the new explainer

In [4]:
from src.explainer.explainer_factory import ExplainerFactory
from src.evaluation.evaluation_metric_factory import EvaluationMetricFactory

class CustomExplainerFactory(ExplainerFactory):

    def __init__(self, explainer_store_path):
        super().__init__(explainer_store_path)

    def get_explainer_by_name(self, explainer_dict, metric_factory : EvaluationMetricFactory) -> Explainer:
        explainer_name = explainer_dict['name']

        # Check if the explainer is DCE Search
        if explainer_name == 'dummy_explainer':
            # Returning the explainer
            return self.get_dummy_explainer(explainer_dict)
        else:
            return super().get_explainer_by_name(explainer_dict, metric_factory)

    def get_dummy_explainer(self, config_dict=None):
        result = DummyExplainer(self._explainer_id_counter, config_dict)
        self._explainer_id_counter += 1
        return result
            


In [5]:
ex_factory = CustomExplainerFactory(ex_store_path)

# The run number is a way to differentiate many runs of the same configurations
eval_manager = EvaluatorManager(config_file_path, run_number=0, 
                                dataset_factory=None, 
                                embedder_factory=None, 
                                oracle_factory=None, 
                                explainer_factory=ex_factory, 
                                evaluation_metric_factory=None)
eval_manager.create_evaluators()
eval_manager.evaluate()

In [6]:
with open(output_file_path, 'r') as rs_json_reader:
                results = rs_json_reader.read()

results

'{"config": {"dataset": {"name": "autism", "parameters": {}}, "oracle": {"name": "asd_custom_oracle", "parameters": {}}, "explainer": {"name": "dummy_explainer", "parameters": {}}, "metrics": [{"name": "graph_edit_distance", "parameters": {}}, {"name": "oracle_calls", "parameters": {}}, {"name": "correctness", "parameters": {}}, {"name": "sparsity", "parameters": {}}, {"name": "fidelity", "parameters": {}}, {"name": "oracle_accuracy", "parameters": {}}]}, "runtime": [0.002505779266357422, 0.001361846923828125, 0.0005340576171875, 0.0012595653533935547, 0.002105236053466797, 0.0014488697052001953, 0.0012538433074951172, 0.0012094974517822266, 0.0022635459899902344, 0.0018124580383300781, 0.0016002655029296875, 0.0013415813446044922, 0.0012216567993164062, 0.0012845993041992188, 0.0010364055633544922, 0.0016109943389892578, 0.0013437271118164062, 0.0018134117126464844, 0.0013954639434814453, 0.0012295246124267578, 0.0015778541564941406, 0.0014688968658447266, 0.005125761032104492, 0.0023

In [7]:
from src.data_analysis.data_analyzer import DataAnalyzer

dtan = DataAnalyzer(output_folder, stats_folder)
dtan.aggregate_data()
dtan.aggregate_runs()
dtan.create_tables_by_oracle_dataset()

In [8]:
import pandas as pd
results_table = pd.read_csv(module_path + '/examples/stats/autism-asd_custom_oracle.csv')
results_table

,Unnamed: 0,explainer,runtime,runtime-std,Graph_Edit_Distance,Graph_Edit_Distance-std,Oracle_Calls,Oracle_Calls-std,Correctness,Correctness-std,Sparsity,Sparsity-std,Fidelity,Fidelity-std,Oracle_Accuracy,Oracle_Accuracy-std
0,0,dce_search,0.072224,0.0,1011.693069,0.0,102.000000,0.0,1.0,0.0,1.311108,0.0,0.544554,0.0,0.772277,0.0
1,1,dummy_explainer,0.001474,0.0,1068.237624,0.0,2.930693,0.0,1.0,0.0,1.384470,0.0,0.544554,0.0,0.772277,0.0
